## Load historical data


In [248]:
import yfinance as yf
import pandas as pd

ticker = yf.Ticker('AAPL')
#df = ticker.history(period='15y', interval="1d")

In [249]:
shares = ticker.shares["BasicShares"].iloc[-1]
shares

16701272000

## Financials

In [250]:
cash = ticker.balance_sheet.T["Cash"].iloc[0]
cash

34940000000.0

In [251]:
netincome_df = ticker.cashflow.T[["Net Income"]].sort_values("")
netincome_df = netincome_df.reset_index()
netincome_df

,,Net Income
0,2018-09-29,5.953100e+10
1,2019-09-28,5.525600e+10
2,2020-09-26,5.741100e+10
3,2021-09-25,9.468000e+10


In [252]:
netincome_df["Growth"] = (netincome_df["Net Income"] - netincome_df["Net Income"].shift())/netincome_df["Net Income"]
netincome_df = netincome_df.fillna(0).iloc[1:, :][["Net Income", "Growth"]]
netincome_df

,Net Income,Growth
1,5.525600e+10,-0.077367
2,5.741100e+10,0.037536
3,9.468000e+10,0.393631


## Params

In [253]:
import math

discount_rate = 0.10
years = 60
maturity = math.floor(years/2)
growth = netincome_df["Growth"].mean(axis=0)
growth_slowdown = 0.05

## Add forcast

In [254]:
for i in range(years):
    row ={"Net Income": netincome_df["Net Income"].iloc[-1] * (growth+1), "Growth": growth}
    rowdf = pd.DataFrame.from_records([row])
    netincome_df = pd.concat([netincome_df, rowdf], ignore_index=True, axis=0)
    if i == maturity:
        growth = -growth_slowdown
    
    if i >= maturity:
        growth = growth * (1 + growth_slowdown)
    else:
        growth = growth * (1 - growth_slowdown)

netincome_df = netincome_df.iloc[3:, :]
netincome_df

,Net Income,Growth
3,1.058459e+11,0.117933
4,1.177046e+11,0.112037
5,1.302325e+11,0.106435
6,1.434007e+11,0.101113
7,1.571754e+11,0.096058
8,1.715184e+11,0.091255
9,1.863876e+11,0.086692
10,2.017380e+11,0.082357
11,2.175219e+11,0.078239
12,2.336898e+11,0.074327


In [255]:
cashflows = netincome_df["Net Income"].to_list()

## DCF

In [256]:
import numpy_financial  as npf

npv = npf.npv(discount_rate, cashflows);
value = npv + cash
forcast_price = value/shares
forcast_price

156.08760480771898